In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import time
from datetime import datetime
import requests
from config import weather_api_key

In [2]:
lats = np.random.uniform(low=-90.000,high=90.000,size=2000)
lngs = np.random.uniform(low=-180.000,high=180.000,size=2000)
lat_lngs = zip(lats,lngs)
lat_lngs

In [3]:
coordinates = list(zip(lats,lngs))

In [4]:
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
        
len(cities)

773

In [5]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
city_data = []

record_count = 1
set_count = 1

In [8]:
for i,city in enumerate(cities):
    
    
    if(i % 50 ==0 and i >= 50):
        set_count +=1
        record_count=1
        time.sleep(60)
        
    city_url = url +'&q=' +city.replace(" ","+")
    
    record_count +=1
    
   
    try:
        city_weather = requests.get(city_url).json()
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        weather_description = city_weather['weather'][0]['description']
        
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        city_data.append({'City': city.title(),
                          'Lat': city_lat,
                          'Lng': city_lng,
                          'Max Temp': city_max_temp,
                          'Humidity': city_humidity,
                          'Cloudiness': city_clouds,
                          'Wind Speed': city_wind,
                          'Country': city_country,
                          'Current Description' : weather_description
                          })
    except:
        pass
    

In [9]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Bradesti,44.4833,23.6333,29.89,77,79,11.77,RO,broken clouds
1,Vangaindrano,-23.3500,47.6000,73.04,85,0,5.06,MG,clear sky
2,Albany,42.6001,-73.9662,8.85,61,100,15.99,US,overcast clouds
3,Half Moon Bay,37.4636,-122.4286,64.08,61,0,1.99,US,clear sky
4,Esperance,-33.8667,121.9000,76.84,49,2,9.01,AU,clear sky


In [11]:
column_order = ['City','Country','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Current Description']
city_data_df = city_data_df[column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Bradesti,RO,44.4833,23.6333,29.89,77,79,11.77,broken clouds
1,Vangaindrano,MG,-23.3500,47.6000,73.04,85,0,5.06,clear sky
2,Albany,US,42.6001,-73.9662,8.85,61,100,15.99,overcast clouds
3,Half Moon Bay,US,37.4636,-122.4286,64.08,61,0,1.99,clear sky
4,Esperance,AU,-33.8667,121.9000,76.84,49,2,9.01,clear sky
5,Chapais,CA,49.7834,-74.8492,-18.62,99,6,2.59,clear sky
6,Airai,TL,-8.9266,125.4092,66.16,80,100,4.38,overcast clouds
7,Fortuna,US,40.5982,-124.1573,58.19,78,0,0.00,clear sky
8,Bredasdorp,ZA,-34.5322,20.0403,65.28,88,5,2.30,clear sky
9,Broome,US,42.2506,-75.8330,10.02,76,75,4.00,broken clouds


In [12]:
#create the output file (CSV)
output_data_file = 'WeatherPy_Database.csv'
#export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label='City_ID')